# Amazon Nova Lite (or can Nova Pro or Nova Premier) Model

In [1]:
import os
import boto3
import json
import base64
from PIL import Image
bedrockruntime = boto3.client('bedrock-runtime', region_name='us-west-2')
novalite = 'us.amazon.nova-lite-v1:0'

# Define Nova Lite prompt function (.csv based)

In [2]:
import pandas as pd
df = pd.read_csv("Ecommerce data.csv")
rows = df.to_dict(orient="records")
format = "\n".join([str(row) for row in rows])
print(format)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


{'Order ID': 'CA-2016-152156', 'Order Date': 42682, 'Ship Date': '11/11/2016', 'Ship Mode': 'Second Class', 'Customer ID': 'CG-12520', 'Customer Name': 'Claire Gute', 'Segment': 'Consumer', 'City': 'Henderson', 'State': 'Kentucky', 'Product ID': 'FUR-BO-10001798', 'Category': 'Furniture', 'Sub-Category': 'Bookcases', 'Product Name': 'Bush Somerset Collection Bookcase', 'Sales': 261.96, 'Quantity': 2, 'Discount': 0.0, 'Profit': 41.9136}
{'Order ID': 'CA-2016-152156', 'Order Date': 42682, 'Ship Date': '11/11/2016', 'Ship Mode': 'Second Class', 'Customer ID': 'CG-12520', 'Customer Name': 'Claire Gute', 'Segment': 'Consumer', 'City': 'Henderson', 'State': 'Kentucky', 'Product ID': 'FUR-CH-10000454', 'Category': 'Furniture', 'Sub-Category': 'Chairs', 'Product Name': 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 'Sales': 731.94, 'Quantity': 3, 'Discount': 0.0, 'Profit': 219.582}
{'Order ID': 'US-2015-108966', 'Order Date': 42288, 'Ship Date': '18/10/2015', 'Ship Mode': 'Stan

In [3]:
def csvprompt(csvpath, prompt, maxTokens, temperature):
    import pandas as pd
    df = pd.read_csv(csvpath)
    rows = df.to_dict(orient="records")
    format = "\n".join([str(row) for row in rows])

    message = [
        {
            "role": "user",
            "content": [
                {
                    "text": prompt + "\n" + format
                }
            ],
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

# Prompt Engineering - Data Analysis

In [4]:
csvpath = "Ecommerce data.csv"
prompt = '''
1. What product name that Budiono Santoso buy?
2. How much was the total sales in the data?
3. How much was the total profit in the data?
4. What product name of the best-selling based quantity?
Answer this question with step-by-step and thinking before answer, without detailed explanation.
'''.strip()
maxTokens = 400
temperature = 0.1
csvresult = csvprompt(csvpath, prompt, maxTokens, temperature)
print(csvresult)

{'output': {'message': {'content': [{'text': "1. **Product Name that Budiono Santoso bought:**\n   - Look for the entry where 'Customer Name' is 'Budiono Santoso'.\n   - The product name in that entry is 'New Paper High Quality'.\n\n2. **Total Sales in the data:**\n   - Sum the 'Sales' values from all entries.\n   - Calculation: 261.96 + 731.94 + 957.5775 + 22.368 + 15.552 + 95.616 + 75.88 + 15.555 = 2270.9495.\n\n3. **Total Profit in the data:**\n   - Sum the 'Profit' values from all entries.\n   - Calculation: 41.9136 + 219.582 + (-383.031) + 2.5164 + 5.4432 + 9.5616 + 35.6636 + 5.4432 = -30.908.\n\n4. **Best-selling product based on quantity:**\n   - Identify the entry with the highest 'Quantity' value.\n   - The entry with the highest quantity is 'Bretford CR4500 Series Slim Rectangular Table' with a quantity of 5.\n   - The product name of the best-selling based on quantity is 'Bretford CR4500 Series Slim Rectangular Table'."}], 'role': 'assistant'}}, 'stopReason': 'end_turn', 'us

In [5]:
answer = csvresult["output"]["message"]["content"][0]["text"]
print(answer)

1. **Product Name that Budiono Santoso bought:**
   - Look for the entry where 'Customer Name' is 'Budiono Santoso'.
   - The product name in that entry is 'New Paper High Quality'.

2. **Total Sales in the data:**
   - Sum the 'Sales' values from all entries.
   - Calculation: 261.96 + 731.94 + 957.5775 + 22.368 + 15.552 + 95.616 + 75.88 + 15.555 = 2270.9495.

3. **Total Profit in the data:**
   - Sum the 'Profit' values from all entries.
   - Calculation: 41.9136 + 219.582 + (-383.031) + 2.5164 + 5.4432 + 9.5616 + 35.6636 + 5.4432 = -30.908.

4. **Best-selling product based on quantity:**
   - Identify the entry with the highest 'Quantity' value.
   - The entry with the highest quantity is 'Bretford CR4500 Series Slim Rectangular Table' with a quantity of 5.
   - The product name of the best-selling based on quantity is 'Bretford CR4500 Series Slim Rectangular Table'.


# Define Nova Lite prompt function (.pdf based)

In [6]:
def pdfprompt(pdfpath, prompt, maxTokens, temperature):
    # Read and base64 encode the image
    with open(pdfpath, "rb") as file:
        docs_bytes = file.read()
        base64_string = base64.b64encode(docs_bytes).decode("utf-8")

    extension = os.path.splitext(pdfpath)[1][1:]
    filename = os.path.splitext(pdfpath)[0]

    # Create the message structure
    message = [
        {
            "role": "user",
            "content": [
                {
                    "document": {
                        "format": extension,
                        "name": filename,
                        "source": {
                            "bytes": base64_string
                        }
                    }
                },
                {
                    "text": prompt
                }
            ],
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

# Prompt Engineering - Recommendation / Decision Making

In [7]:
pdfpath = "Always Winner CV.pdf"

minimal_requirements = '''
Bachelor's or Master's degree in Computer Science, Engineering, Mathematics, or a related field

Min. 1 year of work experience in developing and implementing AI solutions

Proficient in frameworks such as TensorFlow, PyTorch, and programming languages like Python, R, Java, or C++

Skilled in RAG (Retrieval-Augmented Generation) techniques and LLMs (Large Language Models)

Basic knowledge of application development and deployment processes

In-depth understanding of machine learning algorithms, deep learning, and other AI techniques

Experience with data preprocessing and data analysis techniques

Ability to evaluate and improve AI model performance
'''.strip()

preferred_requirements = '''
Experienced with cloud services such as AWS, Google Cloud, or Azure
'''.strip()

In [10]:
prompt = '''
1. Analysis this CV without rewrite the results of the CV extract.
2. Do you recommend this candidate for next step based on {minimum_requirements} and {preferred_requirements}?
'''.strip()
maxTokens = 1000
temperature = 0.1
pdfresult = pdfprompt(pdfpath, prompt, maxTokens, temperature)
print(pdfresult)

{'output': {'message': {'content': [{'text': "Based on the CV provided, Always Winner has a solid background in AI and machine learning, with relevant experience and education. Here's an analysis of their qualifications:\n\nStrengths:\n\nExperience: Always Winner has worked as an AI Engineer at Winner, LLC, and as a Junior AI Engineer at Nerwin, LLC, both in New York City. This indicates a strong professional background in AI.\n\nEducation: They hold a Bachelor of Artificial Intelligence from the University of AI Everywhere, which is a relevant and specialized degree.\n\nTechnical Skills: Always Winner has a broad range of technical skills, including programming languages (Python, Java, C++), machine learning frameworks (TensorFlow, PyTorch, Keras), and cloud platforms (AWS, Azure, Google Cloud). They also have experience with generative AI models and various machine learning techniques.\n\nRecommendation:\n\nMinimum Requirements: Always Winner meets the minimum requirements for the po

In [11]:
answer = pdfresult["output"]["message"]["content"][0]["text"]
print(answer)

Based on the CV provided, Always Winner has a solid background in AI and machine learning, with relevant experience and education. Here's an analysis of their qualifications:

Strengths:

Experience: Always Winner has worked as an AI Engineer at Winner, LLC, and as a Junior AI Engineer at Nerwin, LLC, both in New York City. This indicates a strong professional background in AI.

Education: They hold a Bachelor of Artificial Intelligence from the University of AI Everywhere, which is a relevant and specialized degree.

Technical Skills: Always Winner has a broad range of technical skills, including programming languages (Python, Java, C++), machine learning frameworks (TensorFlow, PyTorch, Keras), and cloud platforms (AWS, Azure, Google Cloud). They also have experience with generative AI models and various machine learning techniques.

Recommendation:

Minimum Requirements: Always Winner meets the minimum requirements for the position, as they have relevant experience in AI and machine

# Define Nova Lite prompt function (.png or .jpg based)

In [12]:
def imageprompt(imagepath, prompt, maxTokens, temperature):
    # Read and base64 encode the image
    with open(imagepath, "rb") as file:
        image_bytes = file.read()
        base64_string = base64.b64encode(image_bytes).decode("utf-8")

    extension = os.path.splitext(imagepath)[1][1:]

    # Create the message structure
    message = [
        {
            "role": "user",
            "content": [
                {
                    "image": {
                        "format": extension,
                        "source": {
                            "bytes": base64_string
                        },
                    }
                },
                {
                    "text": prompt
                }
            ],
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

# Prompt Engineering - Structured Output

In [14]:
imagepath = "receipts.jpg"
prompt = '''
Extract this receipt and write result of extraction with format:
- Store Name :
- Date :
- Total :
'''.strip()
maxTokens = 300
temperature = 0.1
imageresult = imageprompt(imagepath, prompt, maxTokens, temperature)
print(imageresult)

{'output': {'message': {'content': [{'text': '- Store Name : PASTA & CO.\n- Date : 08-12-2023\n- Total : $29.80'}], 'role': 'assistant'}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 1699, 'outputTokens': 36, 'totalTokens': 1735, 'cacheReadInputTokenCount': 0, 'cacheWriteInputTokenCount': 0}}


In [15]:
answer = imageresult["output"]["message"]["content"][0]["text"]
print(answer)

- Store Name : PASTA & CO.
- Date : 08-12-2023
- Total : $29.80


# Prompt Engineering - Content Generation

In [20]:
def contentprompt(prompt, maxTokens, temperature):
    message = [
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

In [21]:
prompt = '''
Create a marketing campaign to attract customers based this data:
- Topic : Cheap landed house in strategic location.
- Target and age : Milenial and Gen Z, between 19 - 40.
- Tone : Detailed informative, invite customers to buy.
'''.strip()
maxTokens = 2000
temperature = 0.1
result = contentprompt(prompt, maxTokens, temperature)
print(result)

{'output': {'message': {'content': [{'text': '### Marketing Campaign: "Your Dream Home Awaits"\n\n#### **Campaign Overview:**\nIntroducing an exclusive marketing campaign designed to attract millennial and Gen Z homebuyers aged 19-40. Our focus is on offering affordable landed houses in prime locations, providing a perfect blend of comfort, convenience, and value.\n\n---\n\n### **1. Campaign Name:**\n**"Your Dream Home Awaits"**\n\n---\n\n### **2. Campaign Objectives:**\n- **Raise Awareness:** Highlight the availability of cheap landed houses in strategic locations.\n- **Generate Interest:** Engage potential buyers with detailed, informative content.\n- **Drive Action:** Encourage immediate inquiries and property viewings.\n\n---\n\n### **3. Target Audience:**\n- **Demographics:** Millennials and Gen Z, aged 19-40.\n- **Psychographics:** Young professionals, families, and individuals seeking affordable housing solutions in desirable areas.\n\n---\n\n### **4. Key Messaging:**\n- **Prima

In [22]:
answer = result["output"]["message"]["content"][0]["text"]
print(answer)

### Marketing Campaign: "Your Dream Home Awaits"

#### **Campaign Overview:**
Introducing an exclusive marketing campaign designed to attract millennial and Gen Z homebuyers aged 19-40. Our focus is on offering affordable landed houses in prime locations, providing a perfect blend of comfort, convenience, and value.

---

### **1. Campaign Name:**
**"Your Dream Home Awaits"**

---

### **2. Campaign Objectives:**
- **Raise Awareness:** Highlight the availability of cheap landed houses in strategic locations.
- **Generate Interest:** Engage potential buyers with detailed, informative content.
- **Drive Action:** Encourage immediate inquiries and property viewings.

---

### **3. Target Audience:**
- **Demographics:** Millennials and Gen Z, aged 19-40.
- **Psychographics:** Young professionals, families, and individuals seeking affordable housing solutions in desirable areas.

---

### **4. Key Messaging:**
- **Primary Message:** "Discover affordable landed houses in prime locations, per

# Prompt Engineering - QnA (Question and Answering)

In [23]:
def qnaprompt(prompt, maxTokens, temperature):
    message = [
        {
            "role": "user",
            "content": [
                {
                    "text": prompt
                }
            ]
        }
    ]

    inf_params = {"maxTokens": maxTokens, "temperature": temperature}

    request = {
        "messages": message,
        "inferenceConfig": inf_params
    }

    response = bedrockruntime.invoke_model(modelId=novalite, body=json.dumps(request))
    model_response = json.loads(response["body"].read())
    return model_response

In [31]:
article = '''
Amazon Bedrock is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies 
like AI21 Labs, Anthropic, Cohere, DeepSeek, Luma, Meta, Mistral AI, poolside (coming soon), Stability AI, TwelveLabs (coming soon),
Writer, and Amazon through a single API, along with a broad set of capabilities you need to build generative AI applications with 
security, privacy, and responsible AI. Using Amazon Bedrock, you can easily experiment with and evaluate top FMs for your use case, 
privately customize them with your data using techniques such as fine-tuning and Retrieval Augmented Generation (RAG), and build agents 
that execute tasks using your enterprise systems and data sources. Since Amazon Bedrock is serverless, you don't have to manage 
any infrastructure, and you can securely integrate and deploy generative AI capabilities into your applications using 
the AWS services you are already familiar with.
'''.strip()
prompt = '''
Based on {article}, I want to asked:
1. Is gemma available in Bedrock?
2. Is Bedrock uses instance type or serverless?
3. What is the function of bedrock with simple explanation?
Answer all questions without detailed explanation.
'''.strip()
maxTokens = 300
temperature = 0.1
result = qnaprompt(prompt, maxTokens, temperature)
print(result)

{'output': {'message': {'content': [{'text': '1. Is gemma available in Bedrock?\n   - Not available\n\n2. Is Bedrock uses instance type or serverless?\n   - Serverless\n\n3. What is the function of bedrock with simple explanation?\n   - Provides a scalable, serverless environment for running applications.'}], 'role': 'assistant'}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 51, 'outputTokens': 58, 'totalTokens': 109, 'cacheReadInputTokenCount': 0, 'cacheWriteInputTokenCount': 0}}


In [32]:
answer = result["output"]["message"]["content"][0]["text"]
print(answer)

1. Is gemma available in Bedrock?
   - Not available

2. Is Bedrock uses instance type or serverless?
   - Serverless

3. What is the function of bedrock with simple explanation?
   - Provides a scalable, serverless environment for running applications.
